# 🛡️ GHAS Code Scanning Demo — CodeQL CLI & GitHub CLI
**Globomantics Robot Fleet Manager** | Modules 5 & 6

This notebook demonstrates CodeQL CLI and GitHub CLI commands for managing code scanning alerts, inspecting results, and triaging vulnerabilities.

In [1]:
import subprocess
import json
import os

REPO = "timothywarner-org/globomantics-robot-fleet"
REPO_DIR = r"C:\github\globomantics-robot-fleet"

# Ensure CodeQL CLI is in PATH for subprocess calls
os.environ["PATH"] = r"C:\codeql-home\codeql" + os.pathsep + os.environ["PATH"]

def run(cmd, shell="pwsh", cwd=REPO_DIR):
    """Run a command and display output with formatting."""
    print(f"\u26a1 Running: {cmd}\n{'\u2500' * 60}")
    result = subprocess.run(
        [shell, "-NoProfile", "-Command", cmd],
        capture_output=True, text=True, cwd=cwd,
        env={**os.environ, "NO_COLOR": "1"}
    )
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.stderr.strip():
        print(f"\u26a0\ufe0f {result.stderr.strip()}")
    print(f"{'\u2500' * 60}")
    return result

def run_gh(cmd, cwd=REPO_DIR):
    """Run a gh/git command directly."""
    print(f"\u26a1 Running: {cmd}\n{'\u2500' * 60}")
    result = subprocess.run(
        cmd, shell=True, capture_output=True, text=True, cwd=cwd,
        env={**os.environ, "NO_COLOR": "1"}
    )
    if result.stdout.strip():
        print(result.stdout.strip())
    if result.stderr.strip():
        print(f"\u26a0\ufe0f {result.stderr.strip()}")
    print(f"{'\u2500' * 60}")
    return result

## 🔧 Pre-flight Checks

Verify that all required CLI tools are installed and authenticated before running demo commands.

In [2]:
# Verify CodeQL CLI is installed
run_gh("codeql --version")

# Verify GitHub CLI is authenticated
run_gh("gh auth status")

# Verify we are in a valid git repo and show current commit
run_gh(f"git -C {REPO_DIR} rev-parse HEAD")

⚡ Running: codeql --version
────────────────────────────────────────────────────────────


CodeQL command-line toolchain release 2.24.2.
Copyright (C) 2019-2026 GitHub, Inc.
Unpacked in: C:\codeql-home\codeql
   Analysis results depend critically on separately distributed query and
   extractor modules. To list modules that are visible to the toolchain,
   use 'codeql resolve packs' and 'codeql resolve languages'.
────────────────────────────────────────────────────────────
⚡ Running: gh auth status
────────────────────────────────────────────────────────────


github.com
  ✓ Logged in to github.com account timothywarner (keyring)
  - Active account: true
  - Git operations protocol: https
  - Token: gho_************************************
  - Token scopes: 'admin:org', 'admin:public_key', 'delete_repo', 'gist', 'repo', 'workflow'
────────────────────────────────────────────────────────────
⚡ Running: git -C C:\github\globomantics-robot-fleet rev-parse HEAD
────────────────────────────────────────────────────────────
234de93bf85d9dd92f2614db9b8003bb075825fa
────────────────────────────────────────────────────────────


CompletedProcess(args='git -C C:\\github\\globomantics-robot-fleet rev-parse HEAD', returncode=0, stdout='234de93bf85d9dd92f2614db9b8003bb075825fa\n', stderr='')

## 🔍 CodeQL Default Setup Configuration

CodeQL **default setup** is GitHub's zero-config option. You don't write a workflow file — GitHub detects the languages, picks the query suite, and runs analysis automatically.

> **💡 GH-500 Exam Tip:** "Default setup" and "default suite" are **different things**. Default setup is the zero-config enablement mode. The default suite is a specific set of queries (fewer, lower noise). Default setup lets you choose *either* the default suite or the security-extended suite.

In [3]:
# Check CodeQL analysis configuration (default setup)
run_gh(f'gh api repos/{REPO}/code-scanning/default-setup --jq "."')

# List recent CodeQL workflow runs
run_gh(f'gh run list --repo {REPO} --workflow=codeql.yml --limit=5')

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/default-setup --jq "."
────────────────────────────────────────────────────────────


{
"languages": [
"actions",
"javascript",
"javascript-typescript",
"python",
"rust",
"typescript"
],
"query_suite": "extended",
"runner_label": null,
"runner_type": "standard",
"schedule": null,
"state": "not-configured",
"threat_model": "remote",
"updated_at": null
}
────────────────────────────────────────────────────────────
⚡ Running: gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=codeql.yml --limit=5
────────────────────────────────────────────────────────────


⚠️ HTTP 404: workflow codeql.yml not found on the default branch (https://api.github.com/repos/timothywarner-org/globomantics-robot-fleet/actions/workflows/codeql.yml)
────────────────────────────────────────────────────────────


CompletedProcess(args='gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=codeql.yml --limit=5', returncode=1, stdout='', stderr='HTTP 404: workflow codeql.yml not found on the default branch (https://api.github.com/repos/timothywarner-org/globomantics-robot-fleet/actions/workflows/codeql.yml)\n')

## 📋 List All Code Scanning Alerts

The `code-scanning/alerts` endpoint returns every alert found by any scanner configured on the repo. Each alert includes the rule ID, tool name, severity, and current state (open or dismissed).

In [4]:
# List all code scanning alerts with key fields
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | {{number, rule: .rule.id, tool: .tool.name, severity: .rule.severity, state: .state}}"'
)

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | {number, rule: .rule.id, tool: .tool.name, severity: .rule.severity, state: .state}"
────────────────────────────────────────────────────────────


{
"number": 7,
"rule": "javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect",
"severity": "warning",
"state": "open",
"tool": "Semgrep OSS"
}
{
"number": 6,
"rule": "javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret",
"severity": "warning",
"state": "open",
"tool": "Semgrep OSS"
}
{
"number": 5,
"rule": "javascript.lang.security.detect-child-process.detect-child-process",
"severity": "error",
"state": "open",
"tool": "Semgrep OSS"
}
{
"number": 4,
"rule": "rust.lang.security.args.args",
"severity": "note",
"state": "open",
"tool": "Semgrep OSS"
}
{
"number": 3,
"rule": "js/code-injection",
"severity": "error",
"state": "open",
"tool": "CodeQL"
}
{
"number": 2,
"rule": "js/insecure-helmet-configuration",
"severity": "error",
"state": "open",
"tool": "CodeQL"
}
{
"number": 1,
"rule": "js/clear-text-cookie",
"severity": "warning",
"state": "open",
"tool": "CodeQL"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | {number, rule: .rule.id, tool: .tool.name, severity: .rule.severity, state: .state}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 7\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"\x1b[m\x1b,\x1b[m\n\x1b"severity"\x1b[m\x1b:\x1b[m \x1b"warning"\x1b[m\x1b,\x1b[m\n\x1b"state"\x1b[m\x1b:\x1b[m \x1b"open"\x1b[m\x1b,\x1b[m\n\x1b"tool"\x1b[m\x1b:\x1b[m \x1b"Semgrep OSS"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 6\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"\x1b[m\x1b,\x1b[m\n\x1b"severity"\x1b[m\x1b:\x1b[m \x1b"warning"\x1b[m\x1b,\x1b[m\n\x1b"state"\x1b[m\x1b:\x1b[m \x1b"open"\x1b[m\x1b,\x1b[m\n\x1b"tool"\x1b[m\x1b:\x1b[m \x1b"Semgrep OSS"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b

## 🔍 Filter Alerts by Severity

Code scanning maps severities as follows:
- **error** = high / critical findings — prioritize these
- **warning** = medium findings
- **note** = low findings

> **💡 GH-500 Exam Tip:** Severity `error` maps to high/critical. `warning` is medium. `note` is low. Triage by severity to focus on what matters most.

In [5]:
# Count of high/critical alerts (severity = error)
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq "[.[] | select(.rule.severity == \\"error\\")] | length"'
)

# List high/critical alerts with rule IDs
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.rule.severity == \\"error\\") | {{number, rule: .rule.id}}"'
)

# Warning-level alerts
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.rule.severity == \\"warning\\") | {{number, rule: .rule.id}}"'
)

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq "[.[] | select(.rule.severity == \"error\")] | length"
────────────────────────────────────────────────────────────


3
────────────────────────────────────────────────────────────
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.rule.severity == \"error\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 5,
"rule": "javascript.lang.security.detect-child-process.detect-child-process"
}
{
"number": 3,
"rule": "js/code-injection"
}
{
"number": 2,
"rule": "js/insecure-helmet-configuration"
}
────────────────────────────────────────────────────────────
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.rule.severity == \"warning\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 7,
"rule": "javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"
}
{
"number": 6,
"rule": "javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"
}
{
"number": 1,
"rule": "js/clear-text-cookie"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.rule.severity == \\"warning\\") | {number, rule: .rule.id}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 7\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 6\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 1\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"js/clear-text-cookie"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## 🔍 Filter Alerts by Tool

When multiple scanners (CodeQL, Semgrep, etc.) report results to the same repo, you can filter by tool name to see which scanner found each issue. This is essential for multi-scanner repos.

In [6]:
# CodeQL alerts only
print("=== CodeQL Alerts ===")
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.tool.name == \\"CodeQL\\") | {{number, rule: .rule.id}}"'
)

# Semgrep alerts only (tool name is "Semgrep OSS" in the API)
print("\n=== Semgrep OSS Alerts ===")
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts '
    f'--jq ".[] | select(.tool.name == \\"Semgrep OSS\\") | {{number, rule: .rule.id}}"'
)

=== CodeQL Alerts ===
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \"CodeQL\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 3,
"rule": "js/code-injection"
}
{
"number": 2,
"rule": "js/insecure-helmet-configuration"
}
{
"number": 1,
"rule": "js/clear-text-cookie"
}
────────────────────────────────────────────────────────────

=== Semgrep OSS Alerts ===
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \"Semgrep OSS\") | {number, rule: .rule.id}"
────────────────────────────────────────────────────────────


{
"number": 7,
"rule": "javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"
}
{
"number": 6,
"rule": "javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"
}
{
"number": 5,
"rule": "javascript.lang.security.detect-child-process.detect-child-process"
}
{
"number": 4,
"rule": "rust.lang.security.args.args"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts --jq ".[] | select(.tool.name == \\"Semgrep OSS\\") | {number, rule: .rule.id}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 7\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.express.security.audit.possible-user-input-redirect.unknown-value-in-redirect"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 6\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.jsonwebtoken.security.jwt-hardcode.hardcoded-jwt-secret"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 5\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"javascript.lang.security.detect-child-process.detect-child-process"\x1b[m\n\x1b}\x1b[m\n\x1b{\x1b[m\n\x1b"number"\x1b[m\x1b:\x1b[m 4\x1b,\x1b[m\n\x1b"rule"\x1b[m\x1b:\x1b[m \x1b"rust.lang.security.args.args"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## 📊 Alert Detail — Show Paths & Data Flow

**Path-problem** queries trace data flow from a **source** (where user input enters) through intermediate nodes to a **sink** (where the dangerous operation occurs). An unbroken path from source to sink is a confirmed true positive.

Key vulnerable patterns in this repo:

| Alert | Type | Show Paths? |
|-------|------|-------------|
| `eval()` code injection | Path-problem | Yes — traces `:format` param to `eval()` |
| lodash merge mass assignment | Path-problem | Yes — traces `req.body` to `_.merge()` |
| Hardcoded session secret | Problem (no path) | No |

> **💡 GH-500 Exam Tip:** Show paths traces source to sink. Use it to validate true positives and explain to developers *why* code is vulnerable, not just *where*.

In [7]:
# Get detailed info for alert #1 (adjust number as needed)
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts/1 '
    f'--jq "{{number, rule: .rule.id, severity: .rule.security_severity_level, '
    f'description: .rule.description, tool: .tool.name, state: .state, html_url: .html_url}}"'
)

# Get alert instances/locations for alert #1
run_gh(
    f'gh api repos/{REPO}/code-scanning/alerts/1/instances '
    f'--jq ".[] | {{ref: .ref, state: .state, location: .location}}"'
)

⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts/1 --jq "{number, rule: .rule.id, severity: .rule.security_severity_level, description: .rule.description, tool: .tool.name, state: .state, html_url: .html_url}"
────────────────────────────────────────────────────────────


{
"description": "Clear text transmission of sensitive cookie",
"html_url": "https://github.com/timothywarner-org/globomantics-robot-fleet/security/code-scanning/1",
"number": 1,
"rule": "js/clear-text-cookie",
"severity": "medium",
"state": "open",
"tool": "CodeQL"
}
────────────────────────────────────────────────────────────
⚡ Running: gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts/1/instances --jq ".[] | {ref: .ref, state: .state, location: .location}"
────────────────────────────────────────────────────────────


{
"location": {
"end_column": 3,
"end_line": 32,
"path": "server.js",
"start_column": 9,
"start_line": 27
},
"ref": "refs/heads/main",
"state": "open"
}
────────────────────────────────────────────────────────────


CompletedProcess(args='gh api repos/timothywarner-org/globomantics-robot-fleet/code-scanning/alerts/1/instances --jq ".[] | {ref: .ref, state: .state, location: .location}"', returncode=0, stdout='\x1b{\x1b[m\n\x1b"location"\x1b[m\x1b:\x1b[m \x1b{\x1b[m\n\x1b"end_column"\x1b[m\x1b:\x1b[m 3\x1b,\x1b[m\n\x1b"end_line"\x1b[m\x1b:\x1b[m 32\x1b,\x1b[m\n\x1b"path"\x1b[m\x1b:\x1b[m \x1b"server.js"\x1b[m\x1b,\x1b[m\n\x1b"start_column"\x1b[m\x1b:\x1b[m 9\x1b,\x1b[m\n\x1b"start_line"\x1b[m\x1b:\x1b[m 27\n\x1b}\x1b[m\x1b,\x1b[m\n\x1b"ref"\x1b[m\x1b:\x1b[m \x1b"refs/heads/main"\x1b[m\x1b,\x1b[m\n\x1b"state"\x1b[m\x1b:\x1b[m \x1b"open"\x1b[m\n\x1b}\x1b[m\n', stderr='')

## ⚡ Alert Triage — Dismiss & Reopen

Not every alert demands immediate remediation. The dismissal workflow lets you document decisions and creates an **audit trail**.

Three dismissal reasons:
1. **False positive** — the scanner got it wrong
2. **Won't fix** — real issue but accepted risk
3. **Used in tests** — intentionally vulnerable test/demo code

> **💡 GH-500 Exam Tip:** Dismissals require documentation. Know all three reasons: false positive, won't fix, used in tests. All create audit trails. Your dismissal comment should be defensible — "I didn't feel like fixing it" won't fly with auditors.

In [8]:
# ======================================================================
# DISMISS AN ALERT
# ======================================================================
# UNCOMMENT the lines below to dismiss — replace alert number as needed
#
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--method PATCH '
#     f'--field state=dismissed '
#     f'--field dismissed_reason="used in tests" '
#     f'--field dismissed_comment="Educational demo repo with intentional vulnerabilities"'
# )
#
# # Verify the dismissal
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--jq "{{number, state, dismissed_reason, dismissed_comment}}"'
# )

# ======================================================================
# RE-OPEN A DISMISSED ALERT
# ======================================================================
# UNCOMMENT the lines below to re-open — replace alert number as needed
#
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--method PATCH '
#     f'--field state=open'
# )
#
# # Verify re-open
# run_gh(
#     f'gh api repos/{REPO}/code-scanning/alerts/1 '
#     f'--jq "{{number, state, dismissed_reason}}"'
# )

print("\u2139\ufe0f  Dismiss/reopen commands are commented out to prevent accidental execution.")
print("   Uncomment the section you need and update the alert number.")

ℹ️  Dismiss/reopen commands are commented out to prevent accidental execution.
   Uncomment the section you need and update the alert number.


## 🚀 Workflow Status & Logs

When troubleshooting, start with the workflow status. Is it passing, failing, or stuck? The `gh` CLI gives you a quick summary. The logs tell you everything — permission failures show 403, build failures show compiler errors, extraction issues list which files could not be processed.

In [9]:
# List recent workflow runs (all workflows)
run_gh(f'gh run list --repo {REPO} --limit=5')

# List CodeQL-specific runs
run_gh(f'gh run list --repo {REPO} --workflow=codeql.yml --limit=5')

# List Semgrep-specific runs
run_gh(f'gh run list --repo {REPO} --workflow=semgrep-analysis.yml --limit=5')

⚡ Running: gh run list --repo timothywarner-org/globomantics-robot-fleet --limit=5
────────────────────────────────────────────────────────────


completed	success	feat: add CodeQL CLI sections to demo notebook with executed outputs	Semgrep Security Analysis	main	push	22238644899	32s	2026-02-20T19:46:00Z
completed	skipped	deps(deps): bump the production-minor-patch group across 1 directory with 25 updates	Semgrep Security Analysis	dependabot/npm_and_yarn/production-minor-patch-402545934f	pull_request	22238329230	1s	2026-02-20T19:36:21Z
completed	failure	deps(deps): bump the production-minor-patch group across 1 directory with 25 updates	Dependency Review	dependabot/npm_and_yarn/production-minor-patch-402545934f	pull_request	22238329214	30s	2026-02-20T19:36:21Z
completed	skipped	deps(deps): bump serialize-javascript from 3.0.0 to 7.0.2	Semgrep Security Analysis	dependabot/npm_and_yarn/serialize-javascript-7.0.2	pull_request	22238219578	1s	2026-02-20T19:33:00Z
completed	failure	deps(deps): bump serialize-javascript from 3.0.0 to 7.0.2	Dependency Review	dependabot/npm_and_yarn/serialize-javascript-7.0.2	pull_request	22238219577	22s

⚠️ HTTP 404: workflow codeql.yml not found on the default branch (https://api.github.com/repos/timothywarner-org/globomantics-robot-fleet/actions/workflows/codeql.yml)
────────────────────────────────────────────────────────────
⚡ Running: gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=semgrep-analysis.yml --limit=5
────────────────────────────────────────────────────────────


completed	success	feat: add CodeQL CLI sections to demo notebook with executed outputs	Semgrep Security Analysis	main	push	22238644899	32s	2026-02-20T19:46:00Z
completed	skipped	deps(deps): bump the production-minor-patch group across 1 directory with 25 updates	Semgrep Security Analysis	dependabot/npm_and_yarn/production-minor-patch-402545934f	pull_request	22238329230	1s	2026-02-20T19:36:21Z
completed	skipped	deps(deps): bump serialize-javascript from 3.0.0 to 7.0.2	Semgrep Security Analysis	dependabot/npm_and_yarn/serialize-javascript-7.0.2	pull_request	22238219578	1s	2026-02-20T19:33:00Z
completed	skipped	deps(deps): bump tar from 4.4.8 to 7.5.8	Semgrep Security Analysis	dependabot/npm_and_yarn/tar-7.5.8	pull_request	22238218433	2s	2026-02-20T19:32:57Z
completed	skipped	deps(deps): bump ws from 5.2.0 to 8.19.0	Semgrep Security Analysis	dependabot/npm_and_yarn/ws-8.19.0	pull_request	22238214430	1s	2026-02-20T19:32:50Z
────────────────────────────────────────────────────────────


CompletedProcess(args='gh run list --repo timothywarner-org/globomantics-robot-fleet --workflow=semgrep-analysis.yml --limit=5', returncode=0, stdout='completed\tsuccess\tfeat: add CodeQL CLI sections to demo notebook with executed outputs\tSemgrep Security Analysis\tmain\tpush\t22238644899\t32s\t2026-02-20T19:46:00Z\ncompleted\tskipped\tdeps(deps): bump the production-minor-patch group across 1 directory with 25 updates\tSemgrep Security Analysis\tdependabot/npm_and_yarn/production-minor-patch-402545934f\tpull_request\t22238329230\t1s\t2026-02-20T19:36:21Z\ncompleted\tskipped\tdeps(deps): bump serialize-javascript from 3.0.0 to 7.0.2\tSemgrep Security Analysis\tdependabot/npm_and_yarn/serialize-javascript-7.0.2\tpull_request\t22238219578\t1s\t2026-02-20T19:33:00Z\ncompleted\tskipped\tdeps(deps): bump tar from 4.4.8 to 7.5.8\tSemgrep Security Analysis\tdependabot/npm_and_yarn/tar-7.5.8\tpull_request\t22238218433\t2s\t2026-02-20T19:32:57Z\ncompleted\tskipped\tdeps(deps): bump ws from 5.

## 🎯 Upload SARIF via CodeQL CLI

External CI systems (Jenkins, Azure DevOps, GitLab CI) can upload SARIF results to GitHub using the CodeQL CLI or the `upload-sarif` action. The `--sarif-category` flag is **required** when multiple scanners report results — without it, uploads overwrite each other.

> **💡 GH-500 Exam Tip:** Third-party scanners use `upload-sarif`, NOT `codeql-action/analyze`. SARIF 2.1.0 is the only supported version. Categories prevent result overwrites — always set `--sarif-category` or the `category` input when using multiple scanners.

---

# 🔬 CodeQL CLI — Local Analysis

The following sections demonstrate using the **CodeQL CLI** directly to create databases, run queries, and analyze results locally. This is the workflow for advanced setup, custom queries, and CI/CD integration outside of GitHub Actions.

> **💡 GH-500 Exam Tip:** The CodeQL CLI is the same engine that powers GitHub's default setup. Running it locally gives you full control over languages, query suites, build modes, and output formats.

## 🔬 CodeQL CLI: Resolve Languages & Packs

Before creating a database, check which languages and query packs are available in your CodeQL bundle.

In [10]:
# What languages does CodeQL support?
print("=== Supported Languages ===")
run_gh("codeql resolve languages")

# What query packs are bundled?
print("\n=== Available Query Packs (first 20) ===")
result = run_gh("codeql resolve packs")

# Show the CodeQL version and install path
print("\n=== CodeQL Version ===")
run_gh("codeql version --format=json")

=== Supported Languages ===
⚡ Running: codeql resolve languages
────────────────────────────────────────────────────────────


actions (C:\codeql-home\codeql\actions)
cpp (C:\codeql-home\codeql\cpp)
csharp (C:\codeql-home\codeql\csharp)
csv (C:\codeql-home\codeql\csv)
go (C:\codeql-home\codeql\go)
html (C:\codeql-home\codeql\html)
java (C:\codeql-home\codeql\java)
javascript (C:\codeql-home\codeql\javascript)
properties (C:\codeql-home\codeql\properties)
python (C:\codeql-home\codeql\python)
ruby (C:\codeql-home\codeql\ruby)
rust (C:\codeql-home\codeql\rust)
swift (C:\codeql-home\codeql\swift)
xml (C:\codeql-home\codeql\xml)
yaml (C:\codeql-home\codeql\yaml)
────────────────────────────────────────────────────────────

=== Available Query Packs (first 20) ===
⚡ Running: codeql resolve packs
────────────────────────────────────────────────────────────


Searching directories specified by `--additional-packs`. All directories have equal priority.
  Searching in:
    No packs were found at this location.
Searching directories specified by `--search-path`. Directories are searched in order.
Searching the root of the CodeQL distribution.
  Searching in:
      C:\codeql-home\codeql
    The following packs were found:
      legacy-upgrades@0.0.0: (query) C:\codeql-home\codeql\legacy-upgrades\qlpack.yml
Searching the parent directory of the CodeQL distribution.
  Searching in:
      C:\codeql-home
    Some packs were hidden by previously found packs. Use --show-hidden-packs to see them.
Searching the local pack cache. This only applies when searching for a particular version of a pack specified in a lock file.
  Searching in: C:\Users\timot\.codeql\packages
  The following packs were found:
    codeql/cpp-queries:
      @0.9.3: (query) C:\Users\timot\.codeql\packages\codeql\cpp-queries\0.9.3\qlpack.yml
      @1.3.9: (query) C:\Users\timot\.c

{
  "productName" : "CodeQL",
  "vendor" : "GitHub",
  "version" : "2.24.2",
  "sha" : "d476ae3155ecd23f92405899f1556b23cd2c8bc4",
  "branches" : [
    "codeql-cli-2.24.2"
  ],
  "copyright" : "Copyright (C) 2019-2026 GitHub, Inc.",
  "unpackedLocation" : "C:\\codeql-home\\codeql",
  "configFileLocation" : "C:\\Users\\timot\\.config\\codeql\\config",
  "configFileFound" : false,
  "overlayVersion" : 4,
  "features" : {
    "analysisSummaryV2Default" : true,
    "buildModeOption" : true,
    "bundleSupportsIncludeDiagnostics" : true,
    "bundleSupportsIncludeLogs" : true,
    "bundleSupportsOverlay" : true,
    "databaseInterpretResultsSupportsSarifRunProperty" : true,
    "featuresInVersionResult" : true,
    "indirectTracingSupportsStaticBinaries" : false,
    "informsAboutUnsupportedPathFilters" : true,
    "supportsPython312" : true,
    "mrvaPackCreate" : true,
    "threatModelOption" : true,
    "traceCommandUseBuildMode" : true,
    "v2ramSizing" : true,
    "mrvaPackCreateMulti

CompletedProcess(args='codeql version --format=json', returncode=0, stdout='{\n  "productName" : "CodeQL",\n  "vendor" : "GitHub",\n  "version" : "2.24.2",\n  "sha" : "d476ae3155ecd23f92405899f1556b23cd2c8bc4",\n  "branches" : [\n    "codeql-cli-2.24.2"\n  ],\n  "copyright" : "Copyright (C) 2019-2026 GitHub, Inc.",\n  "unpackedLocation" : "C:\\\\codeql-home\\\\codeql",\n  "configFileLocation" : "C:\\\\Users\\\\timot\\\\.config\\\\codeql\\\\config",\n  "configFileFound" : false,\n  "overlayVersion" : 4,\n  "features" : {\n    "analysisSummaryV2Default" : true,\n    "buildModeOption" : true,\n    "bundleSupportsIncludeDiagnostics" : true,\n    "bundleSupportsIncludeLogs" : true,\n    "bundleSupportsOverlay" : true,\n    "databaseInterpretResultsSupportsSarifRunProperty" : true,\n    "featuresInVersionResult" : true,\n    "indirectTracingSupportsStaticBinaries" : false,\n    "informsAboutUnsupportedPathFilters" : true,\n    "supportsPython312" : true,\n    "mrvaPackCreate" : true,\n    "t

## 🔬 CodeQL CLI: Create a Database

`codeql database create` extracts source code into a relational database that queries run against. For **interpreted languages** (JavaScript, Python, Ruby), no build step is needed — CodeQL extracts directly from source. For **compiled languages** (C++, Java, Go), you must provide build commands.

> **💡 GH-500 Exam Tip:** JavaScript uses `build-mode: none` — direct extraction, no build step. That's why default setup works seamlessly for JS repos. C++ typically needs `build-mode: manual`.

In [11]:
import shutil

DB_PATH = os.path.join(REPO_DIR, "codeql-db-javascript")

# Clean up any previous database
if os.path.exists(DB_PATH):
    shutil.rmtree(DB_PATH)
    print(f"\U0001f5d1\ufe0f  Removed previous database at {DB_PATH}\n")

# Create a CodeQL database for JavaScript
# --language=javascript covers both JS and TypeScript
# --source-root points to the repo
# No --command needed for interpreted languages
print("\U0001f4e6 Creating CodeQL database for JavaScript...")
print("   This extracts all JS/TS source into a queryable relational database.\n")

run_gh(
    f'codeql database create "{DB_PATH}" '
    f'--language=javascript '
    f'--source-root="{REPO_DIR}" '
    f'--overwrite'
)

# Show database info
if os.path.exists(DB_PATH):
    print("\n\u2705 Database created successfully!")
    run_gh(f'codeql database print-baseline "{DB_PATH}"')

🗑️  Removed previous database at C:\github\globomantics-robot-fleet\codeql-db-javascript

📦 Creating CodeQL database for JavaScript...
   This extracts all JS/TS source into a queryable relational database.

⚡ Running: codeql database create "C:\github\globomantics-robot-fleet\codeql-db-javascript" --language=javascript --source-root="C:\github\globomantics-robot-fleet" --overwrite
────────────────────────────────────────────────────────────


[2026-02-20 13:57:20] [build-stdout] Single-threaded extraction.
[2026-02-20 13:57:23] [build-stdout] Extracting C:\github\globomantics-robot-fleet\views\dashboard.ejs
[2026-02-20 13:57:23] [build-stdout] Done extracting C:\github\globomantics-robot-fleet\views\dashboard.ejs (192 ms)
[2026-02-20 13:57:23] [build-stdout] Extracting C:\github\globomantics-robot-fleet\views\error.ejs
[2026-02-20 13:57:23] [build-stdout] Done extracting C:\github\globomantics-robot-fleet\views\error.ejs (7 ms)
[2026-02-20 13:57:23] [build-stdout] Extracting C:\github\globomantics-robot-fleet\views\layout.ejs
[2026-02-20 13:57:23] [build-stdout] Done extracting C:\github\globomantics-robot-fleet\views\layout.ejs (5 ms)
[2026-02-20 13:57:23] [build-stdout] Extracting C:\github\globomantics-robot-fleet\views\maintenance.ejs
[2026-02-20 13:57:23] [build-stdout] Done extracting C:\github\globomantics-robot-fleet\views\maintenance.ejs (16 ms)
[2026-02-20 13:57:23] [build-stdout] Extracting C:\github\globomantics

Counted a baseline of 252 lines of code for javascript.
────────────────────────────────────────────────────────────


## 🔬 CodeQL CLI: Analyze with Query Suites

`codeql database analyze` runs a query suite against the database and outputs SARIF results. Three built-in suites:

| Suite | Scope | Use Case |
|-------|-------|----------|
| `code-scanning` (default) | Core security | Low noise, high confidence |
| `security-extended` | Broader security | More coverage, some experimental |
| `security-and-quality` | Security + code quality | Maximum coverage |

> **💡 GH-500 Exam Tip:** The `--format=sarifv2.1.0` flag is required for GitHub ingestion. The default output format is NOT SARIF.

In [12]:
SARIF_OUT = os.path.join(REPO_DIR, "codeql-results.sarif")

# Run analysis with the security-extended suite
# Use the pack:suite syntax: codeql/javascript-queries:codeql-suites/javascript-security-extended.qls
print("\U0001f50d Analyzing with security-extended suite...")
print("   This may take 1-3 minutes depending on repo size.\n")

run_gh(
    f'codeql database analyze "{DB_PATH}" '
    f'--format=sarifv2.1.0 '
    f'--output="{SARIF_OUT}" '
    f'codeql/javascript-queries:codeql-suites/javascript-security-extended.qls'
)

# Show result count
if os.path.exists(SARIF_OUT):
    with open(SARIF_OUT, 'r') as f:
        sarif = json.load(f)
    for run_data in sarif.get("runs", []):
        results = run_data.get("results", [])
        tool_name = run_data.get("tool", {}).get("driver", {}).get("name", "?")
        print(f"\n\U0001f4ca Results: {len(results)} findings from {tool_name}")
        for r in results:
            rule_id = r.get("ruleId", "?")
            severity = r.get("level", "?")
            msg = r.get("message", {}).get("text", "")[:80]
            locs = r.get("locations", [])
            file_info = ""
            if locs:
                phys = locs[0].get("physicalLocation", {})
                uri = phys.get("artifactLocation", {}).get("uri", "?")
                line = phys.get("region", {}).get("startLine", "?")
                file_info = f" ({uri}:{line})"
            print(f"   \u26a0\ufe0f  [{severity}] {rule_id}{file_info}")
            print(f"      {msg}")

🔍 Analyzing with security-extended suite...
   This may take 1-3 minutes depending on repo size.

⚡ Running: codeql database analyze "C:\github\globomantics-robot-fleet\codeql-db-javascript" --format=sarifv2.1.0 --output="C:\github\globomantics-robot-fleet\codeql-results.sarif" codeql/javascript-queries:codeql-suites/javascript-security-extended.qls
────────────────────────────────────────────────────────────


CodeQL scanned 2 out of 2 GitHub Actions files and 2 out of 2 JavaScript/TypeScript files in this invocation. Typically CodeQL is configured to analyze a single CodeQL language per invocation, so check other invocations to determine overall coverage information.
⚠️ Running queries.
[1/107] Loaded C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\AngularJS\DisablingSce.qlx.
[2/107] Loaded C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\AngularJS\DoubleCompilation.qlx.
[3/107] Loaded C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\AngularJS\InsecureUrlWhitelist.qlx.
[4/107] Loaded C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Electron\AllowRunningInsecureContent.qlx.
[5/107] Loaded C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Electron\DisablingWebSecurity.qlx.
[6/107] Loaded C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Performance\PolynomialReDoS.qlx.
[7/107] Loaded C:\Users\timot\.codeq

## 🔬 CodeQL CLI: Run a Specific Query

You can also run individual queries instead of full suites. This is useful for:
- Testing custom queries you're developing
- Running a single check against a specific CWE
- Debugging false positives by isolating one rule

In [13]:
# List queries in the security-extended suite
print("=== JavaScript Security-Extended Queries (first 20) ===")
run_gh('codeql resolve queries codeql/javascript-queries:codeql-suites/javascript-security-extended.qls --format=text')

# Run JUST the code injection query (the eval() vulnerability)
SINGLE_SARIF = os.path.join(REPO_DIR, "codeql-single-query.sarif")
print("\n=== Running Single Query: js/code-injection ===")
run_gh(
    f'codeql database analyze "{DB_PATH}" '
    f'--format=sarifv2.1.0 '
    f'--output="{SINGLE_SARIF}" '
    f'codeql/javascript-queries:Security/CWE-094/CodeInjection.ql'
)

if os.path.exists(SINGLE_SARIF):
    with open(SINGLE_SARIF, 'r') as f:
        sarif = json.load(f)
    results = sarif.get("runs", [{}])[0].get("results", [])
    print(f"\n\U0001f3af Code Injection findings: {len(results)}")
    for r in results:
        locs = r.get("locations", [])
        if locs:
            uri = locs[0].get("physicalLocation", {}).get("artifactLocation", {}).get("uri", "?")
            line = locs[0].get("physicalLocation", {}).get("region", {}).get("startLine", "?")
            print(f"   \u274c {uri}:{line} \u2014 {r.get('message', {}).get('text', '')[:100]}")

=== JavaScript Security-Extended Queries (first 20) ===
⚡ Running: codeql resolve queries codeql/javascript-queries:codeql-suites/javascript-security-extended.qls --format=text
────────────────────────────────────────────────────────────


C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\AngularJS\DisablingSce.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\AngularJS\DoubleCompilation.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\AngularJS\InsecureUrlWhitelist.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Electron\AllowRunningInsecureContent.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Electron\DisablingWebSecurity.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Performance\PolynomialReDoS.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Performance\ReDoS.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\RegExp\IdentityReplacement.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Security\CWE-020\IncompleteHostnameRegExp.ql
C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Security\CWE-020\IncompleteUrlSchemeCheck.ql
C:\Users\timot\.codeql\p

⚠️ Running queries.
[1/1] No need to rerun C:\Users\timot\.codeql\packages\codeql\javascript-queries\1.5.4\Security\CWE-094\CodeInjection.ql.
Shutting down query evaluator.
Interpreting results.
────────────────────────────────────────────────────────────

🎯 Code Injection findings: 1
   ❌ server.js:157 — This code execution depends on a [user-provided value](1).


## 🔬 CodeQL CLI: Upload Local Results to GitHub

After running analysis locally, upload the SARIF to GitHub's Security tab using `codeql github upload-results`. This bridges external CI systems with GitHub's unified alert view.

> **💡 GH-500 Exam Tip:** `--sarif-category` is required when uploading from multiple sources. Without it, results from different runs overwrite each other. Each upload source needs its own category namespace.

In [14]:
# Get current commit for the upload
result = subprocess.run("git rev-parse HEAD", shell=True, capture_output=True, text=True, cwd=REPO_DIR)
commit = result.stdout.strip()
print(f"\U0001f4cc Current commit: {commit}\n")

# Show the upload command (ready to uncomment and run)
upload_cmd = (
    f'codeql github upload-results '
    f'--repository={REPO} '
    f'--ref=refs/heads/main '
    f'--commit={commit} '
    f'--sarif="{SARIF_OUT}" '
    f'--sarif-category=codeql-cli-local'
)

print(f"\U0001f4cb Upload command:")
print(f"   {upload_cmd}\n")

# ======================================================================
# UNCOMMENT TO UPLOAD — this pushes results to GitHub Security tab
# ======================================================================
# run_gh(upload_cmd)
# print(f"\n\U0001f517 View results: https://github.com/{REPO}/security/code-scanning")

print("\u2139\ufe0f  Upload command is commented out. Uncomment to push local CodeQL results to GitHub.")
print(f"   SARIF file: {SARIF_OUT}")
print(f"   Category: codeql-cli-local (separate from default setup results)")

📌 Current commit: 234de93bf85d9dd92f2614db9b8003bb075825fa

📋 Upload command:
   codeql github upload-results --repository=timothywarner-org/globomantics-robot-fleet --ref=refs/heads/main --commit=234de93bf85d9dd92f2614db9b8003bb075825fa --sarif="C:\github\globomantics-robot-fleet\codeql-results.sarif" --sarif-category=codeql-cli-local

ℹ️  Upload command is commented out. Uncomment to push local CodeQL results to GitHub.
   SARIF file: C:\github\globomantics-robot-fleet\codeql-results.sarif
   Category: codeql-cli-local (separate from default setup results)


## 🔬 CodeQL CLI: Database Cleanup & Info

Useful utility commands for managing CodeQL databases.

In [15]:
# Show database metadata
print("=== Database Info ===")
run_gh(f'codeql database print-baseline "{DB_PATH}"')

# Show what source files were extracted
print("\n=== Extracted Source Summary ===")
run_gh(f'codeql database index-files --language=javascript "{DB_PATH}" --include-extension=.js')

# Cleanup — remove the database when done (saves ~100MB+)
# Uncomment to clean up:
# import shutil
# shutil.rmtree(DB_PATH, ignore_errors=True)
# print(f"\n\U0001f5d1\ufe0f  Database removed: {DB_PATH}")

print(f"\n\u2139\ufe0f  Database is at: {DB_PATH}")
if os.path.exists(DB_PATH):
    # Calculate size
    total = sum(
        os.path.getsize(os.path.join(dirpath, f))
        for dirpath, dirnames, filenames in os.walk(DB_PATH)
        for f in filenames
    )
    print(f"   Size: {total / (1024*1024):.1f} MB")

=== Database Info ===
⚡ Running: codeql database print-baseline "C:\github\globomantics-robot-fleet\codeql-db-javascript"
────────────────────────────────────────────────────────────


Counted a baseline of 252 lines of code for javascript.
────────────────────────────────────────────────────────────

=== Extracted Source Summary ===
⚡ Running: codeql database index-files --language=javascript "C:\github\globomantics-robot-fleet\codeql-db-javascript" --include-extension=.js
────────────────────────────────────────────────────────────


⚠️ A fatal error occurred: CodeQL database at C:\github\globomantics-robot-fleet\codeql-db-javascript is already finalized.
────────────────────────────────────────────────────────────

ℹ️  Database is at: C:\github\globomantics-robot-fleet\codeql-db-javascript
   Size: 57.0 MB


In [16]:
# ======================================================================
# UPLOAD SARIF FILE VIA CODEQL CLI
# ======================================================================
# UNCOMMENT to upload — requires a .sarif file to exist in the repo root
#
# commit_sha = subprocess.run(
#     ["git", "rev-parse", "HEAD"],
#     capture_output=True, text=True, cwd=REPO_DIR
# ).stdout.strip()
#
# run_gh(
#     f'codeql github upload-results '
#     f'--repository={REPO} '
#     f'--ref=refs/heads/main '
#     f'--commit={commit_sha} '
#     f'--sarif=rust-scan.sarif '
#     f'--sarif-category=semgrep-rust-local',
#     cwd=REPO_DIR
# )

print("\u2139\ufe0f  Upload command is commented out. Uncomment after generating a .sarif file.")
print("   Example: semgrep scan --config p/rust --sarif --output rust-scan.sarif ./rust-telemetry-cli")

ℹ️  Upload command is commented out. Uncomment after generating a .sarif file.
   Example: semgrep scan --config p/rust --sarif --output rust-scan.sarif ./rust-telemetry-cli


## 💡 GH-500 Exam Quick Reference

| Topic | Key Point |
|-------|----------|
| Query suites | default = fewer queries, security-extended = comprehensive |
| Default setup vs default suite | Different things — default setup is zero-config enablement |
| Build modes | `none` (interpreted), `autobuild` (auto-detect), `manual` (explicit) |
| Show paths | Traces source to sink — validates true positives |
| Dismissal reasons | false positive, won't fix, used in tests — all create audit trails |
| SARIF upload | Uses `upload-sarif` action (part of `codeql-action` repo) |
| Categories | REQUIRED for multiple scanners — prevents overwrites |
| Copilot Autofix | Ships with GHAS, no separate Copilot subscription required |
| Copilot Chat | Requires Copilot Enterprise license (separate from GHAS) |
| Troubleshooting | 90% of failures = not explicit enough in configuration |

### Common Failure Scenarios

| Error | Cause | Fix |
|-------|-------|-----|
| Language detection failed | Auto-detection missed a language | Specify `languages:` explicitly in workflow |
| Autobuild failed | Non-standard build system | Use `build-mode: manual` with explicit commands |
| Timeout exceeded | Large codebase | Increase `timeout-minutes`, split into matrix jobs |
| Permission denied (403) | Missing workflow permission | Add `security-events: write` |
| No results returned | Extraction failed silently | Check logs, switch to `security-extended` suite |